In [226]:
import re, json, requests
import pandas as pd
import math

In [238]:
url = 'https://raw.githubusercontent.com/rohitashwachaks/Social-Media-Influencer-analysis/main/Twitter-Influencers/F1_Database/f1_tweets.json'

In [239]:
resp = requests.get(url)
resp_parsed = re.sub(r'^jsonp\d+\(|\)\s+$', '', resp.text)
data = json.loads(resp_parsed)

In [240]:
data = pd.DataFrame(data)

In [243]:
#creating list to store information
user_provide = []
user_receive = []
attention_type = []

In [244]:
for i in range(len(data)):
    
   #reply
    if data.in_reply_to_screen_name[i] != None:
        user_provide.append(data.user[i].get('screen_name'))
        user_receive.append(data.in_reply_to_screen_name[i])
        attention_type.append('reply')
    
    #retweet
    if type(data.retweeted_status[i]) == dict:
        user_provide.append(data.user[i].get('screen_name'))
        user_receive.append(data.retweeted_status[i].get('user').get('screen_name'))
        attention_type.append('RT')
    
    #mention
    if len(data.entities[i].get('user_mentions'))!=0:
        for j in range(len(data.entities[i].get('user_mentions'))):
            user_provide.append(data.user[i].get('screen_name'))
            user_receive.append(data.entities[i].get('user_mentions')[j].get('screen_name'))
            attention_type.append('mention')
        
        

In [245]:
#extract information from tweets 
network_data = pd.DataFrame({'user_provide':user_provide,'user_receive':user_receive,'attention_type':attention_type})


In [ ]:
#Data Transformation for network visualization

In [329]:
network_data = pd.DataFrame({'user_provide':user_provide,'user_receive':user_receive,'attention_type':attention_type})
n = pd.DataFrame(network_data[['user_provide','user_receive','attention_type']].groupby(['user_provide','user_receive'])['attention_type'].agg('count'))
n.to_csv('f1_network_links.csv')

In [331]:
participants = {}
for i in set(network_data.user_provide).union(set(network_data.user_receive)):
    participants[i] = 1
participants = pd.Series(participants)
participants.to_csv('f1_network_participants.csv')

In [332]:
n = pd.crosstab(network_data.user_provide, network_data.user_receive)
idx = n.columns.union(n.index)
n = n.reindex(index = idx, columns=idx, fill_value=0)
n.to_csv('f1_network_adjacency.csv')

In [ ]:
#Creating Network

In [247]:
import networkx as nx 
import matplotlib.pyplot as plt

In [248]:
G = nx.DiGraph(directed=True)

In [254]:
G.add_nodes_from((network_data.user_provide+network_data.user_receive).unique())
G.add_edges_from(tuple(zip(network_data.user_provide,network_data.user_receive)))

In [ ]:
#Drawing Network
nx.draw(G, with_labels=True)
plt.show() 

In [ ]:
#insert the visualization? 

In [258]:
#Calculating network centralities
degree = nx.degree_centrality(G)
betweenness = nx.betweenness_centrality(G)
closeness = nx.closeness_centrality(G)

In [298]:
centrality = pd.DataFrame(degree,index=['degree']).T

In [333]:
centrality['betweenness'] = pd.DataFrame(betweenness,index=['betweenness']).T.betweenness
centrality['closeness'] = pd.DataFrame(closeness,index=['closeness']).T.closeness
centrality = centrality.reset_index().rename(columns={'index':'name'})

In [305]:
for i in range(len(centrality)):
    centrality.loc[i,'sum'] = centrality.loc[i,'degree']+centrality.loc[i,'betweenness']+centrality.loc[i,'closeness']

In [314]:
#Top 10 most important user in the network based on sum of centralities
centrality.sort_values('sum',ascending = False).head(10)

,name,degree,betweenness,closeness,sum
6988,F1,0.132978,0.000025,0.132510,0.265513
6995,redbullracing,0.074846,0.000000,0.072434,0.147280
6987,F1_Jordan,0.055744,0.000000,0.054802,0.110547
7223,McLarenF1,0.006061,0.000000,0.070166,0.076227
10261,lawrobarretto,0.003582,0.000000,0.068921,0.072502
6997,Max33Verstappen,0.034071,0.000000,0.032887,0.066958
7070,fia,0.014235,0.000000,0.017102,0.031337
9761,MrNickKnowles,0.009000,0.000002,0.011704,0.020706
7029,RobLMyers,0.007622,0.000004,0.007925,0.015552
7031,Independent,0.006245,0.000000,0.009257,0.015502
